(Introduce the project)
(Introduce the dataset)

## Gather

(import statements)

In [1]:
import json
import numpy as np
import pandas as pd
import requests
import tweepy
from timeit import default_timer as timer
from tweepy import OAuthHandler

In [2]:
# Load twitter archive file
archive = pd.read_csv('twitter-archive-enhanced.csv')
archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


(load image_predictions.tsv)

In [3]:
# Download image-predictions file
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open('image_predictions.tsv', mode='wb') as file:
    file.write(response.content)

In [4]:
# Import image predictions file
image_predictions = pd.read_csv('image_predictions.tsv', sep='\t')
image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


(use twitter api for json data)
(test on small sample, once working get complete data set once and then comment out)
(temp: load from a saved local file)

In [5]:
# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
consumer_key = 'g2ND7axSfMh4x6NastwdByJQd'
consumer_secret = 'V14qYWCX7aB3nnjoigO1B6hle1D9MOultmA8qc8g2IiQmHQByD'
access_token = '736183273-d5LSGYOp5lxQNPGLN3fTUsEgoNCYElxHIcyVSVHd'
access_secret = 'Q1gRx9PmmufVjl2QyMsKv3sUXJ5RLUn7GaSpugJRv7QXP'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [7]:
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = archive.tweet_id.values
len(tweet_ids)
# Testing REMOVE FOR SUBMISSION
test_tweets = tweet_ids[:20]
len(test_tweets)

20

In [8]:
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in test_tweets:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

1: 892420643555336193
Success
2: 892177421306343426
Success
3: 891815181378084864
Success
4: 891689557279858688
Success
5: 891327558926688256
Success
6: 891087950875897856
Success
7: 890971913173991426
Success
8: 890729181411237888
Success
9: 890609185150312448
Success
10: 890240255349198849
Success
11: 890006608113172480
Success
12: 889880896479866881
Success
13: 889665388333682689
Success
14: 889638837579907072
Success
15: 889531135344209921
Success
16: 889278841981685760
Success
17: 888917238123831296
Success
18: 888804989199671297
Success
19: 888554962724278272
Success
20: 888202515573088257
Fail
10.478039899999999
{888202515573088257: TweepError([{'code': 144, 'message': 'No status found with that ID.'}],)}


In [35]:
# Testing - UPDATE OR REMOVE FOR SUBMISSION
tweets = []
with open('tweet_json.txt') as json_file:
    for line in json_file:
        tweets.append(json.loads(line))
extended_data = pd.DataFrame(tweets)
extended_data.id

0     892420643555336193
1     892177421306343426
2     891815181378084864
3     891689557279858688
4     891327558926688256
5     891087950875897856
6     890971913173991426
7     890729181411237888
8     890609185150312448
9     890240255349198849
10    890006608113172480
11    889880896479866881
12    889665388333682689
13    889638837579907072
14    889531135344209921
15    889278841981685760
16    888917238123831296
17    888804989199671297
18    888554962724278272
Name: id, dtype: int64

## Assess

(Explore each using .info)

#### Quality
##### `twitter_archive_enhanced` table
- x 
- x
- x

##### `image_predictions` table
- x
- x

##### `additional_data` table
- x
- x
- x

#### Tidiness
- x
- x

## Clean

(make copies of datasources)

### (Problem 1)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 2)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 3)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 4)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 5)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 6)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 7)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 8)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 9)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 10)

(Problem: ...)

##### Define
x

##### Code

##### Test

## Storage

(store each table as a csv file with main file called tiwtter_archive_master.csv)

## Analysis

(make at least 3 insights)